### dev_notebook_

- Your development environment...

In [1]:
import pandas as pd
import requests
import json
import duckdb
import re
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from fuzzywuzzy import process
from tqdm import tqdm
import os
from dotenv import load_dotenv




# import numpy as np

In [2]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [3]:
# URL / ENDPOINT
base_url = 'https://datos.madrid.es/egob/'
resource = 'catalogo/300356-0-monumentos-ciudad-madrid.json'

endpoint = base_url + resource
endpoint

'https://datos.madrid.es/egob/catalogo/300356-0-monumentos-ciudad-madrid.json'

In [4]:
# MONUMENTOS MADRID - API Request
monuments = requests.get(endpoint).json()

In [5]:
dict_monuments = {'monument' : [monuments['@graph'][t]['title'] for t in range(len(monuments['@graph']))],
                  'address_mon' : [monuments['@graph'][a]['address']['street-address'] for a in range(len(monuments['@graph']))]}

dict_monuments['lat1'] = []
dict_monuments['lon1'] = []

for i in range(len(monuments['@graph'])):
    try:
        dict_monuments['lat1'].append(monuments['@graph'][i]['location']['latitude'])
        dict_monuments['lon1'].append(monuments['@graph'][i]['location']['longitude'])
    except:
        dict_monuments['lat1'].append(None)
        dict_monuments['lon1'].append(None)
        

#Failed list comprehensions due to lack of 'location' key in some items:
# lat = [bicimad['@graph'][l]['location']['latitude'] for l in range(len(bicimad['@graph']))]
# lon = [bicimad['@graph'][k]['location']['longitude'] for k in range(len(bicimad['@graph']))]

In [6]:
df_monuments = pd.DataFrame(dict_monuments)
df_monuments = df_monuments[df_monuments['lat1'].notna()]
df_monuments = df_monuments[df_monuments['monument'] == 'A los Abuelos']

In [7]:
choices = dict_monuments['monument']

In [8]:
process.extractOne("abuelos", choices)[0]

'A los Abuelos'

In [9]:
# Connexion duckdb

con = duckdb.connect(database='../data/bicimad_jup.db', read_only=False)
df_bicimad = con.execute('SELECT id, name, address, dock_bikes, "geometry.coordinates" FROM bicimad_stations').fetch_df()

In [10]:
df_bicimad = df_bicimad[df_bicimad['dock_bikes'] > 0].reset_index()

In [11]:
regex_loc = '[^, \[\]]+'

In [12]:
bici_loc = [(float(re.findall(regex_loc, df_bicimad['geometry.coordinates'][i])[1]),
             float(re.findall(regex_loc, df_bicimad['geometry.coordinates'][i])[0])) for i in range(len(df_bicimad))]

df_bicimad = df_bicimad.join(pd.DataFrame(bici_loc, columns=['lat2', 'lon2']))
df_bicimad.drop(columns='index', inplace=True)

In [13]:
full_df = df_monuments.merge(df_bicimad, how='cross')

In [14]:
%%time
full_df['pit'] = full_df.apply(lambda x : (x['lat1'] - x['lat2'])**2 + (x['lon1'] - x['lon2'])**2 , axis=1)

CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
bici_min = pd.DataFrame(full_df.groupby(['monument', 'address_mon','lat1','lon1'])['pit'].min())

In [16]:
bici_min = bici_min.reset_index()
mon_bici = bici_min.merge(full_df, how='left').sort_values(by=['monument'])

In [17]:
%%time
mon_bici['Distance (m)'] = mon_bici.apply(lambda x : distance_meters(x['lat1'], x['lon1'], x['lat2'], x['lon2']), axis=1)

CPU times: total: 15.6 ms
Wall time: 117 ms


In [18]:
mon_bici['Walking time (min)'] = mon_bici.apply(lambda x : x['Distance (m)'] / 1000 * 12.5, axis=1)
mon_bici['Distance (m)'], mon_bici['Walking time (min)'] = mon_bici['Distance (m)'].astype(int), mon_bici['Walking time (min)'].astype(int)

In [19]:
#DOTENV - email and password for BiciMAD API

env_path = '../__dotenv__/.env'
load_dotenv(env_path)

email = os.getenv('MAIL')
pw = os.getenv('PASSWORD')

In [31]:
# URL / ENDPOINT / BiciMAD

url_login = 'https://openapi.emtmadrid.es/v2/mobilitylabs/user/login/'
url_stations = 'https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'
headers_login = {'email': email, 'password' : pw}

# Access Token request

token = requests.get(url_login, headers=headers).json()['data'][0]['accessToken']

In [32]:
#Station list request

bici_json = requests.get(url_stations, headers={'accessToken' : token}).json()

In [33]:
bici_json

{'code': '00',
 'description': '358 bases recovered',
 'datetime': '2023-04-14T16:07:59.394067',
 'data': [{'activate': 1,
   'address': 'Calle Enrique Garcia Alvarez, 5,',
   'dock_bikes': 3,
   'free_bases': 24,
   'geometry': {'type': 'Point', 'coordinates': [-3.6140607, 40.3804482]},
   'id': 2187,
   'light': 0,
   'name': '438 - Calle Enrique Garcia Alvarez, 5',
   'no_available': 0,
   'number': '438',
   'reservations_count': 0,
   'total_bases': 27,
   'virtualDelete': False},
  {'activate': 1,
   'address': 'Paseo de la Chopera,33,Comunidad de Madrid España',
   'dock_bikes': 1,
   'free_bases': 24,
   'geometry': {'type': 'Point', 'coordinates': [-3.7004, 40.395]},
   'id': 2205,
   'light': 0,
   'name': '267 - Paseo de la Chopera, 33',
   'no_available': 0,
   'number': '267',
   'reservations_count': 0,
   'total_bases': 25,
   'virtualDelete': False},
  {'activate': 1,
   'address': 'Calle Rafaela Ybarra, 22,Comunidad de Madrid España',
   'dock_bikes': 3,
   'free_bases